CIFAR-10 이미지 분류를 위한 ResNet 구현

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10

In [2]:
NUM_CLASSES = 10

In [3]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data()
print("train_X shape : ", train_X.shape)
print(test_X.shape[0],"test samples")

train_X shape :  (50000, 32, 32, 3)
10000 test samples


In [4]:
input_shape = train_X.shape[1:]

train_X = train_X / 255.0
test_X = test_X / 255.0

In [5]:
inputs = Input(shape=input_shape)

In [6]:
conv = Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")
x = conv(inputs)
x = BatchNormalization()(x)

conv = Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", kernel_regularizer=l2(1e-4))
y = conv(x)
y = BatchNormalization()(y)

x = keras.layers.add([x,y])
x = Activation("relu")(x)

conv = Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", kernel_regularizer=l2(1e-4))
y = conv(x)
y = BatchNormalization()(y)

conv = Conv2D(filters=32, kernel_size=3, padding="same", kernel_regularizer=l2(1e-4))
y = conv(y)
y = BatchNormalization()(y)

x = keras.layers.add([x,y])
x = Activation("relu")(x)

x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)

y = Dense(512, activation="relu")(y)
outputs = Dense(NUM_CLASSES, activation="softmax")(y)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 32)           896       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 32)           128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv2d_1 (Conv2D)           (None, 32, 32, 32)           9248      ['batch_normalization[0][0

In [11]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam

EPOCHS = 100
BATCH_SIZE = 200
save_dir = os.path.join(os.getcwd(), "saved_models")
model_name = "cifar10_model-{epoch:03d}-{val_accuracy:.4f}.h5"

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath, monitor="val_accuracy", save_best_only=True, verbose=1)

In [12]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1
    print("Learning rate : ", lr)
    return lr

In [13]:
lr_scheduler = LearningRateScheduler(lr_schedule)
my_callbacks = [checkpoint, lr_scheduler]
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])
history = model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=my_callbacks)

Learning rate :  0.001
Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 1.3695 - accuracy: 0.5141
Epoch 1: val_accuracy improved from -inf to 0.10020, saving model to C:\Users\KOSA\AI\saved_models\cifar10_model-001-0.1002.h5
250/250 [==============================] - 98s 384ms/step - loss: 1.3695 - accuracy: 0.5141 - val_loss: 3.4284 - val_accuracy: 0.1002 - lr: 0.0010
Learning rate :  0.001
Epoch 2/100


C:\Users\KOSA\anaconda3\envs\tf2.14\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - ETA: 0s - loss: 0.9812 - accuracy: 0.6566
Epoch 2: val_accuracy improved from 0.10020 to 0.54510, saving model to C:\Users\KOSA\AI\saved_models\cifar10_model-002-0.5451.h5
250/250 [==============================] - 100s 400ms/step - loss: 0.9812 - accuracy: 0.6566 - val_loss: 1.2762 - val_accuracy: 0.5451 - lr: 0.0010
Learning rate :  0.001
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.8493 - accuracy: 0.7061
Epoch 3: val_accuracy improved from 0.54510 to 0.66490, saving model to C:\Users\KOSA\AI\saved_models\cifar10_model-003-0.6649.h5
250/250 [==============================] - 97s 388ms/step - loss: 0.8493 - accuracy: 0.7061 - val_loss: 0.9698 - val_accuracy: 0.6649 - lr: 0.0010
Learning rate :  0.001
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 0.7631 - accuracy: 0.7358
Epoch 4: val_accuracy improved from 0.66490 to 0.70810, saving model to C:\Users\KOSA\AI\saved_models\cifar10_model-004-